In [ ]:
# Adjustable Privacy - create_obfuscated_dataset.ipynb
# - Use an obfuscator to obfuscate dataset and create a new one.
# - Uses image dataset (CelebA).
# - Load a specific trained obfuscator model (from google drive).
# - It creates a zip file of obfuscated dataset and saves on google drive and locally. (structured like original celeba files)
# - You can manage notebook parameters in parser block

In [ ]:
# Imports
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
from torch import optim
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.utils.data import random_split
from torchvision import datasets, transforms, models
import torchvision.utils as vutils
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import itertools
import random
import shutil
from zipfile import ZipFile
import os
from tqdm import tqdm
from collections import OrderedDict
import time
from math import floor
import argparse

In [ ]:
# Parser
parser = argparse.ArgumentParser(description='Adjustable Privacy - Use an obfuscator to obfuscate dataset and create a new one. '
                                 + 'Uses image dataset (CelebA). '
                                 + 'Load a specific trained obfuscator model (from google drive). '
                                 + 'It creates a zip file of obfuscated dataset and saves on google drive and locally. (structured like original celeba files) ')

parser.add_argument('--model_number', type=int, required=True, help = 'The epoch number of desired obfuscator model (model number)')
parser.add_argument('--target_index', type=int, required=True, help = 'gender(20), smiling(31), open mouth(21), and high chickbone(19)')
parser.add_argument('--load_path', type=str, required=True, help = 'Full path on your google drive to load model from. Like "drive/MyDrive/adjustable-privacy/Models/CelebA-G-S-Obfuscator/"')
parser.add_argument('--use_g', required=True, help = 'Accepts "True" or "False". Activate g function or not.')
parser.add_argument('--lambda_v', type=int, default = -3000, help = 'Value of lambda (only when use_g=True)')
parser.add_argument('--noise', type=int, default = 0, help = 'Value of noise coeficient.')
parser.add_argument('--version_name', type=str, default = "", help = 'a short name specify dataset like:"GS-model0-g3000-f20" ')
parser.add_argument('--save_path', type=str, required=True, help = 'Full path on your google drive to save obfuscated dataset. Like "drive/MyDrive/adjustable-privacy/Datasets/ObfuscatedCelebA/"')
parser.add_argument('--download_dataset', default = False, help = 'Accepts "True" or "False". Download CelebA dataset or use CelebA shared directory.')
parser.add_argument('--dataset_path', type=str, default = "", help = '(if --download_dataset=False) Full path on your google drive to CelebA shared directory shortcut. Like "drive/MyDrive/adjustable-privacy/Datasets/CelebA/"')

command_string = "--model_number 183" \
" --target_index 20" \
" --load_path drive/MyDrive/adjustable-privacy/Models/CelebA-G-S-Obfuscator/" \
" --use_g True" \
" --lambda_v -3000" \
" --noise 35" \
" --version_name GS-model183-g3000-f35" \
" --save_path drive/MyDrive/adjustable-privacy/Datasets/ObfuscatedCelebA/" \
" --download_dataset False" \
" --dataset_path drive/MyDrive/adjustable-privacy/Datasets/CelebA/"

args = parser.parse_args(command_string.split())

In [ ]:
# Hyper parameters:
isFirstRun = False
lastRunEpochNumber = args.model_number
manual_seed = 20
image_size = 64
use_whole_dataset = True
usage_percent = 1.0
learning_rate = 0.001 #0.2
batch_size = 1
celeba_male_index = 20
celeba_smiling_index = 31
celeba_mouth_open_index = 21
celeba_high_cheekbone_index = 19
using_index = args.target_index
saving_path = args.load_path
# Number of workers for dataloader
workers = 2
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
# Number of GPUs available.
ngpu = 1
# Size of feature maps in encoder
nef = 64
# Size of feature maps in decoder
ndf = 64
# Number of channels in the training images. For color images this is 3
nc = 3
# Number of training epochs
num_epochs = lastRunEpochNumber
data_dir = 'celeba'
download_dataset = args.download_dataset=='True'
dataset_folder_path = args.dataset_path

# creating modified dataset:
m_version = args.version_name
on_drive_modified_dataset_saving_path = args.save_path + m_version + '/'
on_fly_modified_dataset_saving_path = 'modifiedDatasets/' + m_version + '/'

use_g = args.use_g=='True'
g_eff_val = args.lambda_v
miu = 0
coef_for_var = args.noise
load_m_dataset = True
create_zip_from_m_dataset = True
delete_m_dataset_folder = False
zip_and_copy_to_drive = True

suffling_main_train_data = False
suffling_modified_train_data = False

In [ ]:
# Check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [ ]:
# Mount google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# getting dataset ready from shared directory
if not download_dataset:
    dataset_zip_path = dataset_folder_path + '/Img/img_align_celeba.zip'
    list_eval_partition_path = dataset_folder_path + '/Eval/list_eval_partition.txt'
    identity_celeba_path = dataset_folder_path + '/Anno/identity_CelebA.txt'
    list_attr_celeba_path = dataset_folder_path + '/Anno/list_attr_celeba.txt'
    list_bbox_celeba_path = dataset_folder_path + '/Anno/list_bbox_celeba.txt'
    list_landmarks_align_celeba_path = dataset_folder_path + '/Anno/list_landmarks_align_celeba.txt'

    try:
      os.mkdir(data_dir)
      print("data folder created successfully")
    except OSError as e:
      print("Error: %s" % (e.strerror))

    shutil.copyfile(dataset_zip_path, data_dir + r'/img_align_celeba.zip')
    shutil.copyfile(list_eval_partition_path, data_dir + r'/list_eval_partition.txt')
    shutil.copyfile(identity_celeba_path, data_dir + r'/identity_CelebA.txt')
    shutil.copyfile(list_attr_celeba_path, data_dir + r'/list_attr_celeba.txt')
    shutil.copyfile(list_bbox_celeba_path, data_dir + r'/list_bbox_celeba.txt')
    shutil.copyfile(list_landmarks_align_celeba_path, data_dir + r'/list_landmarks_align_celeba.txt')

    try:
        shutil.rmtree(data_dir + r'/img_align_celeba')
        print("old unzipped directory removed successfully")
    except OSError as e:
        print("Error: %s" % (e.strerror))

    archive = data_dir + r'/img_align_celeba.zip'
    with ZipFile(archive, 'r') as zip:
       zip.extractall(data_dir)

data folder created successfully
Error: No such file or directory


In [ ]:
# make saving path dir
try:
    os.makedirs(saving_path)
    print("saving_path directory created successfully")
except OSError as e:
    print("Error: %s" % (e.strerror))

Error: File exists


In [ ]:

# Define transforms
train_transforms = transforms.Compose([transforms.Resize(image_size),
                                       transforms.CenterCrop(image_size),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(image_size),
                                      transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [ ]:
# Function - use some percent of data
def shorten_dataset(dataset, usage_percent=1.0):
  len_used = floor(len(dataset)*usage_percent)
  len_not_used = len(dataset) - len_used
  used_dataset, not_used_dataset = random_split(dataset, [len_used, len_not_used], generator=torch.Generator().manual_seed(manual_seed))
  return used_dataset

In [ ]:

# Load Datas
train_set = datasets.CelebA(root='', download=download_dataset, split='train', target_type=["attr", "identity"], transform=train_transforms)
test_set = datasets.CelebA(root='', download=download_dataset, split='test', target_type=["attr", "identity"], transform=test_transforms)
valid_set = datasets.CelebA(root='', download=download_dataset, split='valid', target_type=["attr", "identity"], transform=test_transforms)

# shorten Dataset
if not use_whole_dataset:
  train_set = shorten_dataset(train_set, usage_percent)

# DataLoader
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=suffling_main_train_data, num_workers=workers)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=workers)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, num_workers=workers)


In [ ]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
# Encoder Model
class Encoder(nn.Module):
    def __init__(self, ngpu):
        super(Encoder, self).__init__()
        self.ngpu = ngpu
        
        # input is nc x 64 x 64 
        self.conv1 = nn.Conv2d(nc, nef, 4, 2, 1, bias=False)
        self.actv1 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 32 x 32
        self.conv2 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor2 = nn.BatchNorm2d(nef)
        self.actv2 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 16 x 16
        self.conv3 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor3 = nn.BatchNorm2d(nef)
        self.actv3 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 8 x 8
        self.conv4 = nn.Conv2d(nef, nef * 2, 4, 2, 1, bias=False)
        self.bnor4 = nn.BatchNorm2d(nef * 2)
        self.actv4 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef*2) x 4 x 4
        # shaping would be here: nef*2 x 4 x 4 -> 2048
        # state size. 2048
        self.fllc5 = nn.Linear(nef*2*4*4, nef*1*4*4)
        self.actv5 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 1024

        # split features: 1024 -> 1022 + 2
        # classifier:
        self.fllc_main_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_main_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_main_features1 = nn.Dropout(p=0.5)
        self.fllc_main_features2 = nn.Linear(nef*1*4*4, nef*4)
        self.actv_main_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_main_features2 = nn.Dropout(p=0.5)
        self.fllc_main_features3 = nn.Linear(nef*4, nef)
        self.actv_main_features3 = nn.LeakyReLU(0.2, inplace=True)
        self.fllc_main_features4 = nn.Linear(nef, 2)
        self.actv_main_features4 = nn.LogSoftmax(dim=1)
        # other features
        self.fllc_other_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_other_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_other_features1 = nn.Dropout(p=0.5)
        self.fllc_other_features2 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_other_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_other_features2 = nn.Dropout(p=0.5)
        self.fllc_other_features3 = nn.Linear(nef*1*4*4, nef*1*4*4 - 2)
        self.actv_other_features3 = nn.LeakyReLU(0.2, inplace=True)
        # aggregate features for output

    def forward(self, x):
        # Part 1:
        x = self.conv1(x)
        x = self.actv1(x)
        x = self.conv2(x)
        x = self.bnor2(x)
        x = self.actv2(x)
        x = self.conv3(x)
        x = self.bnor3(x)
        x = self.actv3(x)
        x = self.conv4(x)
        x = self.bnor4(x)
        x = self.actv4(x)
        # flatten
        x = torch.flatten(x, start_dim = 1)
        # Part 2:
        x = self.fllc5(x)
        x = self.actv5(x)
        # first classifier: 
        y1 = self.fllc_main_features1(x)
        y1 = self.actv_main_features1(y1)
        y1 = self.dropout_main_features1(y1)
        y1 = self.fllc_main_features2(y1)
        y1 = self.actv_main_features2(y1)
        y1 = self.dropout_main_features2(y1)
        y1 = self.fllc_main_features3(y1)
        y1 = self.actv_main_features3(y1)
        y1 = self.fllc_main_features4(y1)
        y1 = self.actv_main_features4(y1)
        # other features
        y3 = self.fllc_other_features1(x) 
        y3 = self.actv_other_features1(y3)
        y3 = self.dropout_other_features1(y3)
        y3 = self.fllc_other_features2(y3) 
        y3 = self.actv_other_features2(y3)
        y3 = self.dropout_other_features2(y3)
        y3 = self.fllc_other_features3(y3) 
        y3 = self.actv_other_features3(y3)
        return y1, y3

In [ ]:
# Decoder Model
class Decoder(nn.Module):
    def __init__(self, ngpu):
        super(Decoder, self).__init__()
        self.ngpu = ngpu
        
        # input size is 1024
        self.fllc6 = nn.Linear(nef*1*4*4, ndf*2*4*4)
        self.actv6 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 2048
        # shaping would be here: 2048 -> ndf*2 x 4 x 4
        # state size. (ndf*2) x 4 x 4
        self.cnvt7 = nn.ConvTranspose2d( ndf*2, ndf, 4, 2, 1, bias=False)
        self.bnor7 = nn.BatchNorm2d(ndf)
        self.actv7 = nn.ReLU(True)
        # state size. (ndf) x 8 x 8
        self.cnvt8 = nn.ConvTranspose2d(ndf, ndf, 4, 2, 1, bias=False)
        self.bnor8 = nn.BatchNorm2d(ndf)
        self.actv8 = nn.ReLU(True)
        # state size. (ndf) x 16 x 16
        self.cnvt9 = nn.ConvTranspose2d( ndf, ndf, 4, 2, 1, bias=False)
        self.bnor9 = nn.BatchNorm2d(ndf)
        self.actv9 = nn.ReLU(True)
        # state size. (ndf) x 32 x 32
        self.cnvt10 = nn.ConvTranspose2d( ndf, nc, 4, 2, 1, bias=False)
        self.actv10 = nn.Sigmoid() # nn.Tanh()
        # state size. (nc) x 64 x 64 

    def forward(self, x):
        x = self.fllc6(x)
        x = self.actv6(x)
        x = x.view(batch_size, ndf*2, 4, 4)
        x = self.cnvt7(x)
        x = self.bnor7(x)
        x = self.actv7(x)
        x = self.cnvt8(x)
        x = self.bnor8(x)
        x = self.actv8(x)
        x = self.cnvt9(x)
        x = self.bnor9(x)
        x = self.actv9(x)
        x = self.cnvt10(x)
        x = self.actv10(x)
        return x

In [ ]:
# AE Model + Noise
class AEModel(nn.Module):
    def __init__(self, ngpu, mode='train', miu=0, coef_for_var=0, g_eff_val=-10000):
        super(AEModel, self).__init__()
        self.ngpu = ngpu
        self.g_eff_val = g_eff_val
        self.miu = miu
        self.coef_for_var = coef_for_var
        self.mode = mode
        self.encoder = Encoder(ngpu).to(device)
        self.decoder = Decoder(ngpu).to(device)

    def tune_noise(self, miu=0, coef_for_var=0, g_eff_val=-10000):
        self.miu = miu
        self.coef_for_var = coef_for_var
        self.g_eff_val = g_eff_val

    def change_mode(self, mode='train'):
        self.mode = mode

    def add_noise(self, nodes):
      with torch.no_grad():
        var = (self.coef_for_var) * (torch.mean(nodes).item())
        noise = self.miu + (var) * torch.randn(nodes.size())
        noise = noise.to(device)
        nodes.add_(noise)
        return nodes

    def change_lbl(self, nodes, lbls):
      with torch.no_grad():
        lbls[lbls == 0] = self.g_eff_val
        lbls[lbls == 1] = 0
        nodes = lbls
        return nodes

    def forward(self, x, y1_real_lbl=[]):
        y1, y3 = self.encoder(x)
        if self.mode=='use':
            if use_g:
              y1 = self.change_lbl(y1, y1_real_lbl)
            y3 = self.add_noise(y3)
        y = torch.cat((y1, y3), 1)
        x = self.decoder(y)
        return x, y1

In [ ]:
# Create the AE
netAE = AEModel(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netAE = nn.DataParallel(netAE, list(range(ngpu)))

In [ ]:
# total parameters
total_params = sum(p.numel() for p in netAE.parameters())
print(f"{total_params:,} total parameters.")

9,204,032 total parameters.


In [ ]:
# Function - Save:
def save_model(name, number, model, res):
  checkpoint = {'res': res,
                'state_dict': model.state_dict()}
  torch.save(checkpoint, saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth')
  return True

In [ ]:
# Function - Load:
def load_model(name, number, model, device):

  checkpoint = torch.load(saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth', map_location=device)
  res = checkpoint['res']
  model.load_state_dict(checkpoint['state_dict'])
  return {'model':model,
          'res':res}

In [ ]:
# Load Checkpoint:
ae_load = load_model('ae', lastRunEpochNumber, netAE, device)
train_losses = ae_load['res']['train_losses']
valid_losses = ae_load['res']['valid_losses']
y1_train_losses = ae_load['res']['y1_train_losses']
y1_valid_losses = ae_load['res']['y1_valid_losses']
test_mse = ae_load['res']['test_mse']
test_psnr = ae_load['res']['test_psnr']
test_y1_acc = ae_load['res']['test_y1_acc']
last_epoch = ae_load['res']['epoch_number']

In [ ]:
def extract_targets(labels):
    main_labels = labels[0][:, using_index]
    reverse_labels = torch.add(1, -main_labels)
    main_target = torch.cat((reverse_labels.view(batch_size,1), main_labels.view(batch_size,1)), 1).float()
    main_target = main_target.to(device)
    return main_target

In [ ]:
# Create new dataset folder:

m_celeba_dir = on_fly_modified_dataset_saving_path + 'celeba/'
img_celeba_dir = on_fly_modified_dataset_saving_path + 'celeba/img_align_celeba/'
try:
    os.mkdir('modifiedDatasets')
except OSError as e:
    print("Error: %s" % (e.strerror))
try:
    os.mkdir(on_fly_modified_dataset_saving_path)
except OSError as e:
    print("Error: %s" % (e.strerror))
try:
    os.mkdir(m_celeba_dir)
except OSError as e:
    print("Error: %s" % (e.strerror))
try:
    os.mkdir(img_celeba_dir)
except OSError as e:
    print("Error: %s" % (e.strerror))

try:
    shutil.copyfile(list_eval_partition_path, m_celeba_dir + r'list_eval_partition.txt')
    shutil.copyfile(identity_celeba_path, m_celeba_dir + r'identity_CelebA.txt')
    shutil.copyfile(list_attr_celeba_path, m_celeba_dir + r'list_attr_celeba.txt')
    shutil.copyfile(list_bbox_celeba_path, m_celeba_dir + r'list_bbox_celeba.txt')
    shutil.copyfile(list_landmarks_align_celeba_path, m_celeba_dir + r'list_landmarks_align_celeba.txt')
    print("label files copied successfully")
except OSError as e:
    print("Error: %s" % (e.strerror))

label files copied successfully


In [ ]:
# Function save image:
from torchvision.utils import save_image

def store_single_disk(image, file_name):
    save_image(image, img_celeba_dir + file_name)

In [ ]:
def convert_dataset(my_loader):
    prog_bar = tqdm(enumerate(my_loader), total=len(my_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            main_target = extract_targets(labels)
            output, y1 = netAE.forward(inputs, main_target)
            for j in range(batch_size):
                store_single_disk(output[j,:,:,:], my_loader.dataset.filename[i*batch_size + j])

In [ ]:
# using model to obfuscate dataset
netAE.change_mode('use')
netAE.eval()
netAE.tune_noise(miu, coef_for_var, g_eff_val)
print("Converting train images...\n")
convert_dataset(train_loader)
print("\nConverting valid images...")
convert_dataset(valid_loader)
print("\nConverting test images...")
convert_dataset(test_loader)

Converting train images...



100%|██████████| 162770/162770 [16:55<00:00, 160.31it/s]


Converting valid images...



100%|██████████| 19867/19867 [02:04<00:00, 160.21it/s]


Converting test images...



100%|██████████| 19962/19962 [02:02<00:00, 162.75it/s]


In [ ]:
# load modified dataset:
if load_m_dataset:
    m_transforms = transforms.Compose([transforms.ToTensor()])
    m_train_set = datasets.CelebA(root=on_fly_modified_dataset_saving_path, download=False, split='train', target_type=["attr", "identity"], transform=m_transforms)
    m_valid_set = datasets.CelebA(root=on_fly_modified_dataset_saving_path, download=False, split='valid', target_type=["attr", "identity"], transform=m_transforms)
    m_test_set = datasets.CelebA(root=on_fly_modified_dataset_saving_path, download=False, split='test', target_type=["attr", "identity"], transform=m_transforms)
    m_train_loader = torch.utils.data.DataLoader(m_train_set, batch_size=batch_size, shuffle=suffling_modified_train_data, num_workers=workers, drop_last=True)
    m_valid_loader = torch.utils.data.DataLoader(m_valid_set, batch_size=batch_size, num_workers=workers, drop_last=True)
    m_test_loader = torch.utils.data.DataLoader(m_test_set, batch_size=batch_size, num_workers=workers, drop_last=True)

In [ ]:
# zip new pics folder & delete other files
if create_zip_from_m_dataset:
    shutil.make_archive(on_fly_modified_dataset_saving_path + 'img_align_celeba', 'zip', img_celeba_dir)

if delete_m_dataset_folder:
    try:
        shutil.rmtree(m_celeba_dir)
        print("modified dataset directory removed successfully")
    except OSError as e:
        print("Error: %s" % (e.strerror))

In [ ]:
# zip celeba folder and copy to drive
if zip_and_copy_to_drive:
  try:
    os.mkdir(on_drive_modified_dataset_saving_path)
  except OSError as e:
    print("Error: %s" % (e.strerror))
  zipped_address = shutil.make_archive(on_drive_modified_dataset_saving_path + 'celeba', 'zip', m_celeba_dir)

Error: No such file or directory
